# TODO

-  ~~ pass batch instead of singal sequence entity~~
- test on some toy example.

- attention mask

- implement positional embeddings (maybe in another notebook)

sources: 

https://jalammar.github.io/illustrated-transformer/

https://auro-227.medium.com/writing-a-custom-layer-in-pytorch-14ab6ac94b77

https://github.com/karpathy/minGPT/blob/master/mingpt/model.py

In [1]:
# %load_ext autoreload
# %autoreload 2

In [262]:
import torch

### SelfAttention

In [281]:
import torch
from torch import nn


class Attention(torch.nn.Module):
    def __init__(self, emb_size, kdim=None, vdim=None):
        """
        emb_size - size of input embedding vector
        by default w_size(aka k_dim, v_dim equal to emb_size) in https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html#torch.nn.MultiheadAttention
        
        """
        super().__init__()
        self.emb_size  = emb_size
        
        # Inner dimensions of Q and K matrices are equal, dimension of V might be different. 
        # ...but by deafault all of them are equal to emb_size
        self.kdim  = kdim or emb_size
        self.vdim  = vdim or emb_size
#         self.W_Q = nn.Parameter(torch.Tensor(self.emb_size, self.kdim))
#         self.W_K = nn.Parameter(torch.Tensor(self.emb_size, self.kdim))
#         self.W_V = nn.Parameter(torch.Tensor(self.emb_size, self.vdim))
        self.W_Q = nn.Linear(self.emb_size, self.kdim)
        self.W_K = nn.Linear(self.emb_size, self.kdim)
        self.W_V = nn.Linear(self.emb_size, self.vdim)
        
    def forward(self, X: torch.Tensor):
        """
        The first step is to calculate the Query, Key, and Value matrices. 
        We do that by packing our embeddings into a matrix X, and multiplying it by the weight 
        matrices we’ve trained (WQ, WK, WV).
        
        - Every row in the X matrix corresponds to a word in the input sentence. 
        
        """
#         Q = torch.matmul(X, self.W_Q)
#         K = torch.matmul(X, self.W_K)
#         V = torch.matmul(X, self.W_V)
        Q = self.W_Q(X)
        K = self.W_K(X)
        V = self.W_V(X)
        numerator = torch.matmul(Q, torch.transpose(K, 1, 2))
        denominator = (self.kdim)**0.5
        
        fraction = torch.softmax(numerator/denominator, axis=-1)
        self_attention = torch.matmul(fraction, V)
        return self_attention
    
emb_size = 20
kdim = 12
vdim = 11
layer = Attention(emb_size=emb_size, kdim=kdim, vdim=vdim)

In [282]:
seq_len = 27
batch_size=100

X = torch.Tensor(1, seq_len, emb_size).random_()
layer(X).shape

torch.Size([1, 27, 11])

### MultiHeadAttention

todo implement general attention version of this not just self attention

In [285]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, n_heads, emb_size, vdim=None, kdim=None):
        super().__init__()
        self.n_heads = n_heads
        self.emb_size  = emb_size
        self.kdim  = kdim or emb_size
        self.vdim  = vdim or emb_size
        assert (self.vdim % self.n_heads)==0, 'emb_size must be divisible by n_heads'
#         self.W_0 = nn.Parameter(torch.Tensor(self.emb_size, self.vdim))
        self.W_0 = nn.Linear(self.vdim, self.vdim, bias=False)
        self.heads = [Attention(self.emb_size, vdim=self.vdim//self.n_heads, 
                                kdim=self.kdim) for i in range(self.n_heads)]
        self.ffn = nn.ReLU(nn.Linear(10, 10))
        
    def forward(self, X: torch.Tensor):
        attentions = []
        # TODO this can and should be done in parallel
        for head in self.heads:
            z_i = head(X)
            attentions.append(z_i)
        Z = torch.concatenate(attentions, axis=-1)
        multihead_attention = self.W_0(Z)
        output = self.ffn(multihead_attention)
        return output

    
n_heads = 3
emb_size = 33
multihead_layer = MultiHeadAttention(n_heads=n_heads,  emb_size=emb_size)#, vdim=30)

seq_len = 47
batch_size=50
X = torch.Tensor(batch_size, seq_len, emb_size).random_()
multihead_layer(X).shape

torch.Size([50, 47, 33])

In [261]:
# t = torch.nn.MultiheadAttention(embed_dim=40, num_heads=3, vdim=3)
# seq_len = 101
# emb_size=40

# X = torch.Tensor(seq_len, emb_size).random_()

# Q, K, V = X, X, X[:, ]

# t(Q, K, V)[0].shape